In [26]:
!pip install fuzzywuzzy python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 44.3 MB/s eta 0:00:00


In [29]:
import pandas as pd
import numpy as np
import re
import spacy
from transformers import pipeline
from transformers import BartTokenizer, BartForConditionalGeneration
from fuzzywuzzy import fuzz
import warnings
warnings.filterwarnings('ignore')

In [23]:
from google.colab import files

## **Data_loading**

In [34]:
df=pd.read_csv('/content/Synthetic_data_HRTECH.csv')

In [6]:
df.head()

,interview_id,name,age,gender,role_applied,industry_skills,jd,interview_date,transcript
0,INT1001,Kabir Patel,37,Female,HR Executive,"employee engagement, communication, recruitmen...","Manage recruitment, employee engagement initia...",2023-07-08,Kabir Patel: Good morning! Thank you for this ...
1,INT1002,Esha Kapoor,24,Male,HR Executive,"employee engagement, communication, recruitmen...","Manage recruitment, employee engagement initia...",2024-09-01,"Esha Kapoor: Hi, thanks for having me today.\n..."
2,INT1003,Kiara Nambiar,25,Other,Data Analyst,"data analysis, sql, excel, visualization, stat...","Analyze business data, build dashboards and re...",2025-06-04,"Kiara Nambiar: Hi, thanks for having me today...."
3,INT1004,Aisha Bhandari,36,Female,HR Executive,"employee engagement, communication, recruitmen...","Manage recruitment, employee engagement initia...",2023-12-07,"Aisha Bhandari: Hi, thanks for having me today..."
4,INT1005,Tara Joshi,27,Female,Marketing Manager,"branding, campaigns, digital marketing, analyt...","Lead marketing campaigns, drive brand awarenes...",2023-02-17,"Tara Joshi: Good afternoon, thank you for taki..."


In [7]:
df['transcript'][0]

"Kabir Patel: Good morning! Thank you for this opportunity.\nInterviewer: Thanks for joining — let's begin.\nInterviewer: What are your key strengths?\nKabir Patel: My strengths are employee engagement and communication.\nInterviewer: Can you walk me through a challenging problem you solved?\nCandidate: My last project involved communication and cross-functional coordination.\nInterviewer: Tell me about a recent technical/project you worked on.\nKabir Patel: My last project involved communication and cross-functional coordination.\nInterviewer: What motivates you at work?\nKabir Patel: I want to deepen my expertise and lead a team of engineers/marketers.\nInterviewer: Do you have any questions for us?\nCandidate: I look forward to hearing from you. Thank you.\nInterviewer: Describe your role in your last project.\nCandidate: My so last project involved communication and cross-functional coordination.\nInterviewer: Tell me about a recent technical/project you worked on.\nKabir Patel: I 

## **clean transcript**

In [71]:
def clean_transcript(text, candidate_name):
    text = str(text) if pd.notna(text) else ""

    if not text:
        return ""
    text = text.replace("Interviewer:", "").replace("Candidate:", "")
    text = re.sub(rf"{re.escape(candidate_name)}\s*[:\-]", "", text, flags=re.IGNORECASE)

    fillers = [
        "Sure, well,", "for sharing that", "Good to know", "That's helpful",
        "Could you elaborate on that", "How did your team respond to your decision",
        "Let's start with some questions", "To begin", "Great question","Tell me about"
        "for this opportunity", "Alright", "Got it","can you tell me about yourself","for joining","let's begin",
        " Do you have", "anything to ask us?","questions for us"
    ]
    for f in fillers:
        text = re.sub(rf"{re.escape(f)}[., ]*", "", text, flags=re.IGNORECASE)


    greetings = [
        "hello", "good morning", "good afternoon", "good evening",
        "thank you", "thanks", "nice to meet you", "hope you are doing well",
        "pleasure to be here", "thank you for having me","You're welcome"
    ]
    for g in greetings:
        text = re.sub(rf"\b{g}\b", "", text, flags=re.IGNORECASE)


    text = re.sub(r"\b(What|Why|How|Tell me|Can you)\b.*?\?", "", text)
    text = re.sub(r"[!?,.]{2,}", ". ", text)
    text = re.sub(r"\.{2,}", ". ", text)
    text = re.sub(r"\s+", " ", text).strip()

    return text

def final_clean_for_summary(text):

    text = re.sub(r"\b(\w+)-\1\b", r"\1", text)
    text = re.sub(r"(you know|uh|sure|well|great,|let's move|tell me about|let's begin|! for joining).*?,,!",
                  "", text, flags=re.IGNORECASE)
    text = re.sub(r"(tell me|describe your|which tools|any \?|you know|for this opportunity\.?)", "", text, flags=re.IGNORECASE)
    text = re.sub(r"[\?\!]", "", text).strip()

    sentences = []
    for s in text.split("."):
        s = s.strip()
        if s and s not in sentences:
            sentences.append(s)
    text = ". ".join(sentences)

    text = re.sub(r"\s+", " ", text).strip()
    return text


In [72]:
df['clean_transcript'] = df.apply(lambda row: clean_transcript(row['transcript'], row['name']), axis=1)

In [74]:
df['clean_transcript'][0]

'! for this opportunity. — My strengths are employee engagement and communication. My last project involved communication and cross-functional coordination. Tell me about a recent technical/project you worked on. My last project involved communication and cross-functional coordination. I want to deepen my expertise and lead a team of engineers/marketers. any ? I look forward to hearing from you. . Describe your role in your last project. My so last project involved communication and cross-functional coordination. Tell me about a recent technical/project you worked on. I well led a project where we used employee engagement to deliver a solution that improved metrics. I so take hands-on courses and apply them to projects. I take hands-on courses and apply them to projects. I you know worked on projects using employee engagement, communication, recruitment. Which tools and technologies do you use? That concludes our questions. Yes, I would like to know more about the team I’d be working w

In [11]:
df['clean_transcript']=df['clean_transcript'].apply(final_clean_for_summary)

In [12]:
df['clean_transcript'][0]

'! for this opportunity. — My strengths are employee engagement and communication. My last project involved communication and cross-functional coordination. Tell me about a recent technical/project you worked on. I want to deepen my expertise and lead a team of engineers/marketers. any ? I look forward to hearing from you. Describe your role in your last project. My so last project involved communication and cross-functional coordination. I well led a project where we used employee engagement to deliver a solution that improved metrics. I so take hands-on courses and apply them to projects. I take hands-on courses and apply them to projects. I you know worked on projects using employee engagement, communication, recruitment. Which tools and technologies do you use? That concludes our questions. Yes, I would like to know more about the team I’d be working with'

## **Summarize**


In [82]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


model_name = "philschmid/bart-large-cnn-samsum"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [86]:
def llm_recruiter_summary(text):
    prompt = """
You are an HR Recruiter. Summarize the candidate’s profile from the following interview transcript.
Focus only on:
- Key technical skills (e.g., Python, SQL, HR, Marketing, Project Management)
- Soft skills (e.g., communication, leadership, teamwork)
- Project experience or achievements
- Career goals or motivation

Do NOT include questions, fillers, stutters, 'you know', 'uh', or repeated lines.
Write in 4-5 clean recruiter-style sentences.

Transcript:
"""
    input_text = prompt + text[:3000]

    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=1024)

    summary_ids = model.generate(
        inputs,
        max_length=200,
        min_length=80,
        num_beams=6,
        no_repeat_ngram_size=3,
        length_penalty=1.3,
        early_stopping=True
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [87]:
text="""! for this opportunity.
— My strengths are employee engagement and communication.
 My last project involved communication and cross-functional coordination.
 Tell me about a recent technical/project you worked on.
  I want to deepen my expertise and lead a team of engineers/marketers.
  any ? I look forward to hearing from you. Describe your role in your last project.
   My so last project involved communication and cross-functional coordination.
   I well led a project where we used employee engagement to deliver a solution that improved metrics.
   I so take hands-on courses and apply them to projects.
   I take hands-on courses and apply them to projects.
    I you know worked on projects using employee engagement, communication, recruitment.
    Which tools and technologies do you use? That concludes our questions.
     Yes, I would like to know more about the team I’d be working with"""

In [88]:
res=llm_recruiter_summary(text)
print(res)

The candidate's strengths are employee engagement and communication. His last project involved communication and cross-functional coordination. He wants to deepen his expertise and lead a team of engineers/marketers. He would like to know more about the team he would be working with. He also wants to know about the tools and technologies he uses. The candidate is looking for an HR Recruiter.


In [89]:
df['summary'] = df['clean_transcript'].apply(llm_recruiter_summary)

In [90]:
def clean_summary(summary):

    summary = re.sub(r"[^.]*\?", "", summary)

    summary = re.sub(r"(He would like to know.*?|He also wants to know.*?|He is looking for.*)", "", summary)

    summary = re.sub(r"\s+", " ", summary).strip()

    return summary

In [91]:
df['summary']=df['summary'].apply(clean_summary)

In [92]:
df['clean_transcript'][0]

'! for this opportunity. — My strengths are employee engagement and communication. My last project involved communication and cross-functional coordination. Tell me about a recent technical/project you worked on. My last project involved communication and cross-functional coordination. I want to deepen my expertise and lead a team of engineers/marketers. any ? I look forward to hearing from you. . Describe your role in your last project. My so last project involved communication and cross-functional coordination. Tell me about a recent technical/project you worked on. I well led a project where we used employee engagement to deliver a solution that improved metrics. I so take hands-on courses and apply them to projects. I take hands-on courses and apply them to projects. I you know worked on projects using employee engagement, communication, recruitment. Which tools and technologies do you use? That concludes our questions. Yes, I would like to know more about the team I’d be working w

In [93]:
df['summary'][0]

"The candidate's strengths are employee engagement and communication. Her last project involved communication and cross-functional coordination. She wants to deepen her expertise and lead a team of engineers/marketers. She would like to know more about the team she would be working with. She is interested in working as an HR Recruiter. She has worked on projects using employee engagement, communication, recruitment and recruitment tools."

In [94]:
df.to_csv('summarized.csv')


In [95]:
files.download('summarized.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Sentiment--Key Word Matching**


In [97]:
df=pd.read_csv('/content/summarized.csv')

In [156]:
global_skills = [
    "python", "java", "c++", "sql", "excel", "power bi", "tableau",
    "machine learning", "deep learning", "nlp","django","pandas",
    "communication", "leadership", "teamwork",
    "git", "docker", "api", "cloud",
    "Jenkins for CI/CD",
    "algorithms", "data structures",
    "campaigns", "brand awareness", "digital marketing","pipeline",
    "seo", "branding", "digital marketing", "analytics",
    "recruitment", "employee engagement", "policy implementation",
    "agile", "scrum", "project management","detail-oriented",
    "communication", "leadership", "teamwork", "problem solving", "employee engagement",
    "collaboration", "stakeholder management","cross-functional coordination"

]


In [157]:
role_skill_map = {
    "Software Engineer": ["python", "java", "git", "api", "algorithms", "data structures"],
    "Data Analyst": ["sql", "excel", "power bi", "data visualization", "statistics", "python"],
    "Marketing Manager": ["campaigns", "brand awareness", "digital marketing", "seo", "content strategy"],
    "HR Executive": ["recruitment", "employee engagement", "onboarding", "policy implementation", "communication"],
    "Project Manager": ["agile", "scrum", "stakeholders", "risk management", "planning"],
    "AI ML Engineer": ["machine learning", "deep learning", "tensorflow", "pytorch", "nlp", "model deployment","ml","dl"]
}


In [158]:
def extract_skills(summary):
    detected = []
    for i in global_skills:
        if i in summary.lower():
          detected.append(i)
    return list(set(detected))

In [159]:
df['key_skills']=df['summary'].apply(extract_skills)

In [160]:
df['clean_transcript'][3]

"Hi, for having me today. Great to meet you. We'll go through some questions. I take hands-on courses and apply them to projects. I have hands-on experience with employee engagement and communication. I use Git for version control and Jenkins for CI/CD. ? I have around 3 years of experience focusing on employee engagement, communication. I enjoy problem-solving and collaborating with teams. Describe your role in your last project. I I mean led a project where we used employee engagement to deliver a solution that improved metrics. Describe your role in your last project. I developed a tool using employee engagement that automated a lot of manual work. I coordinate closely with cross-functional stakeholders to deliver results. I am detail-oriented and I communicate well with stakeholders. Tell me about a recent technical/project you worked on. I have hands-on experience with employee engagement and communication. That concludes our questions. No, . I look forward to hearing from you."

In [161]:
df['summary'][3]

'Interview questions focus on technical skills, soft skills, project experience, career goals and motivation. The candidate has around 3 years of experience focusing on employee engagement, communication and problem-solving. She developed a tool using employee engagement that automated a lot of manual work. She is detail-oriented and she communicates well with stakeholders. She has hands-on experience with employee engagement and communication.'

In [162]:
df['key_skills'][3]

['communication', 'detail-oriented', 'employee engagement']

In [136]:
df.head(5)

,Unnamed: 0.1,Unnamed: 0,interview_id,name,age,gender,role_applied,industry_skills,jd,interview_date,transcript,clean_transcript,summary,key_skills
0,0,0,INT1001,Kabir Patel,37,Female,HR Executive,"employee engagement, communication, recruitmen...","Manage recruitment, employee engagement initia...",2023-07-08,Kabir Patel: Good morning! Thank you for this ...,! for this opportunity. — My strengths are emp...,The candidate's strengths are employee engagem...,"[communication, cross-functional coordination,..."
1,1,1,INT1002,Esha Kapoor,24,Male,HR Executive,"employee engagement, communication, recruitmen...","Manage recruitment, employee engagement initia...",2024-09-01,"Esha Kapoor: Hi, thanks for having me today.\n...","Hi, for having me today. Great to meet you. We...",The candidate has around 5 years of experience...,"[communication, git, detail-oriented, employee..."
2,2,2,INT1003,Kiara Nambiar,25,Other,Data Analyst,"data analysis, sql, excel, visualization, stat...","Analyze business data, build dashboards and re...",2025-06-04,"Kiara Nambiar: Hi, thanks for having me today....","Hi, for having me today. . I I mean take hands...","In the interview transcript, focus on the cand...",[git]
3,3,3,INT1004,Aisha Bhandari,36,Female,HR Executive,"employee engagement, communication, recruitmen...","Manage recruitment, employee engagement initia...",2023-12-07,"Aisha Bhandari: Hi, thanks for having me today...","Hi, for having me today. Great to meet you. We...","Interview questions focus on technical skills,...","[communication, detail-oriented, employee enga..."
4,4,4,INT1005,Tara Joshi,27,Female,Marketing Manager,"branding, campaigns, digital marketing, analyt...","Lead marketing campaigns, drive brand awarenes...",2023-02-17,"Tara Joshi: Good afternoon, thank you for taki...",", for taking the time. . I well enjoy mentorin...","In the interview transcript, focus only on the...","[git, branding, campaigns]"


## **check --key skills**

In [169]:
def check_skills_in_summary(candidate_skills, required_skills_str, threshold=50):
    matched = 0
    required_skills = [skill.strip().lower() for skill in required_skills_str.split(',')]

    if not required_skills:
        return "Neutral"

    for req_skill in required_skills:
        for cand_skill in candidate_skills:
            if fuzz.partial_ratio(req_skill, cand_skill.lower()) > 70:
                matched += 1
                break

    total_required_skills = len(required_skills)
    match_percent = (matched / total_required_skills) * 100

    if match_percent > threshold:
        return "Positive"
    elif match_percent == threshold:
        return "Neutral"
    else:
        return "Negative"

In [170]:
df["JD_match"] = df.apply(
    lambda row: check_skills_in_summary(row["key_skills"], row["industry_skills"]),
    axis=1
)

In [171]:
df['JD_match'].value_counts()

,count
JD_match,
Negative,81
Positive,19


## **Red flag detection**


In [172]:
red_flag_phrases = [
    "i don't know", "not sure", "um", "uh", "you know",
    "blame", "issue with manager", "problem with team",
    "i didn't like", "i hate", "no idea", "confused"
]


In [173]:
def detect_red_flags(text):
    flags = []
    lower_text = text.lower()

    for phrase in red_flag_phrases:
        if phrase in lower_text:
            flags.append(phrase)

    if flags:
        return f"Red Flag  - {', '.join(flags)}"
    else:
        return "No Red Flag"


In [174]:
df["Red_Flag"] = df["summary"].apply(detect_red_flags)


In [175]:
df

,Unnamed: 0.1,Unnamed: 0,interview_id,name,age,gender,role_applied,industry_skills,jd,interview_date,transcript,clean_transcript,summary,key_skills,JD_match,Red_Flag
0,0,0,INT1001,Kabir Patel,37,Female,HR Executive,"employee engagement, communication, recruitmen...","Manage recruitment, employee engagement initia...",2023-07-08,Kabir Patel: Good morning! Thank you for this ...,! for this opportunity. — My strengths are emp...,The candidate's strengths are employee engagem...,"[communication, cross-functional coordination,...",Positive,No Red Flag
1,1,1,INT1002,Esha Kapoor,24,Male,HR Executive,"employee engagement, communication, recruitmen...","Manage recruitment, employee engagement initia...",2024-09-01,"Esha Kapoor: Hi, thanks for having me today.\n...","Hi, for having me today. Great to meet you. We...",The candidate has around 5 years of experience...,"[communication, git, detail-oriented, employee...",Negative,No Red Flag
2,2,2,INT1003,Kiara Nambiar,25,Other,Data Analyst,"data analysis, sql, excel, visualization, stat...","Analyze business data, build dashboards and re...",2025-06-04,"Kiara Nambiar: Hi, thanks for having me today....","Hi, for having me today. . I I mean take hands...","In the interview transcript, focus on the cand...",[git],Negative,"Red Flag - uh, you know"
3,3,3,INT1004,Aisha Bhandari,36,Female,HR Executive,"employee engagement, communication, recruitmen...","Manage recruitment, employee engagement initia...",2023-12-07,"Aisha Bhandari: Hi, thanks for having me today...","Hi, for having me today. Great to meet you. We...","Interview questions focus on technical skills,...","[communication, detail-oriented, employee enga...",Negative,No Red Flag
4,4,4,INT1005,Tara Joshi,27,Female,Marketing Manager,"branding, campaigns, digital marketing, analyt...","Lead marketing campaigns, drive brand awarenes...",2023-02-17,"Tara Joshi: Good afternoon, thank you for taki...",", for taking the time. . I well enjoy mentorin...","In the interview transcript, focus only on the...","[git, branding, campaigns]",Positive,"Red Flag - uh, you know"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,95,INT1096,Ritika Saxena,41,Male,AI/ML Engineer,"machine learning, python, deep learning, model...","Develop and deploy machine learning models, pe...",2024-10-26,Ritika Saxena: Good morning! Thank you for thi...,! for this opportunity. . I led a project wher...,Interviewer is looking for an HR Recruiter. Th...,"[python, machine learning]",Positive,"Red Flag - uh, you know"
96,96,96,INT1097,Vivaan Verma,35,Female,Data Analyst,"data analysis, sql, excel, visualization, stat...","Analyze business data, build dashboards and re...",2024-12-09,Vivaan Verma: Good morning! Thank you for this...,! for this opportunity. . I you know developed...,"In the interview transcript, focus on key tech...",[],Negative,Red Flag - um
97,97,97,INT1098,Neha Nambiar,22,Female,Marketing Manager,"branding, campaigns, digital marketing, analyt...","Lead marketing campaigns, drive brand awarenes...",2023-03-14,Neha Nambiar: Good morning! Thank you for this...,! for this opportunity. Great to meet you. We'...,Focus only on key technical skills and soft sk...,"[branding, campaigns]",Negative,"Red Flag - um, uh, you know"
98,98,98,INT1099,Anaya Bose,32,Male,Software Engineer,"python, java, problem-solving, data structures...",Design and implement scalable software solutio...,2025-07-02,"Anaya Bose: Hi, thanks for having me today.\nI...","Hi, for having me today. . ? I well am a Softw...","In the interview transcript, focus only on the...","[python, java]",Negative,"Red Flag - uh, you know"


In [176]:
df['Red_Flag'].value_counts()

,count
Red_Flag,
No Red Flag,62
"Red Flag - uh, you know",26
Red Flag - um,9
"Red Flag - um, uh, you know",2
Red Flag - you know,1


## **Final_data**

In [201]:
df_final = df[(df['JD_match'] == 'Positive') & (df['Red_Flag'] == 'No Red Flag')]


In [202]:
df_final.shape

(12, 16)

In [203]:
df_final=df_final.drop(columns=['Unnamed: 0'])

In [204]:
df_final=df_final.reset_index(drop=True)

In [206]:
df_final

,Unnamed: 0.1,interview_id,name,age,gender,role_applied,industry_skills,jd,interview_date,transcript,clean_transcript,summary,key_skills,JD_match,Red_Flag
0,0,INT1001,Kabir Patel,37,Female,HR Executive,"employee engagement, communication, recruitmen...","Manage recruitment, employee engagement initia...",2023-07-08,Kabir Patel: Good morning! Thank you for this ...,! for this opportunity. — My strengths are emp...,The candidate's strengths are employee engagem...,"[communication, cross-functional coordination,...",Positive,No Red Flag
1,13,INT1014,Myra Kulkarni,36,Female,AI/ML Engineer,"machine learning, python, deep learning, model...","Develop and deploy machine learning models, pe...",2025-03-13,"Myra Kulkarni: Good afternoon, thank you for t...",", for taking the time. . Tell me about a recen...",Interviewer is looking for an HR Recruiter. Th...,"[python, machine learning]",Positive,No Red Flag
2,17,INT1018,Saanvi Saxena,44,Female,AI/ML Engineer,"machine learning, python, deep learning, model...","Develop and deploy machine learning models, pe...",2024-10-15,Saanvi Saxena: Good morning! Thank you for thi...,! for this opportunity. — Tell me about a rece...,"In the interview transcript, focus on key tech...","[python, machine learning, leadership]",Positive,No Red Flag
3,25,INT1026,Saanvi Bhatt,42,Female,Marketing Manager,"branding, campaigns, digital marketing, analyt...","Lead marketing campaigns, drive brand awarenes...",2023-07-02,"Saanvi Bhatt: Hi, thanks for having me today.\...","Hi, for having me today. — Sometimes I get too...",The candidate is an HR Recruiter with experien...,"[cross-functional coordination, git, branding,...",Positive,No Red Flag
4,39,INT1040,Saanvi Rao,33,Male,Marketing Manager,"branding, campaigns, digital marketing, analyt...","Lead marketing campaigns, drive brand awarenes...",2025-07-04,"Saanvi Rao: Hi, thanks for having me today.\nI...","Hi, for having me today. Great to meet you. We...",An HR Recruiter is looking for a candidate wit...,"[git, branding, campaigns]",Positive,No Red Flag
5,49,INT1050,Riya Khan,31,Male,Marketing Manager,"branding, campaigns, digital marketing, analyt...","Lead marketing campaigns, drive brand awarenes...",2023-06-16,"Riya Khan: Hi, thanks for having me today.\nIn...","Hi, for having me today. . I developed a tool ...",An HR Recruiter is interviewing for a position...,"[cross-functional coordination, git, branding,...",Positive,No Red Flag
6,53,INT1054,Sara Patel,32,Male,Marketing Manager,"branding, campaigns, digital marketing, analyt...","Lead marketing campaigns, drive brand awarenes...",2024-11-18,Sara Patel: Good morning! Thank you for this o...,! for this opportunity. . My so last project i...,The candidate's last project involved campaign...,"[cross-functional coordination, git, branding,...",Positive,No Red Flag
7,54,INT1055,Rohan Agarwal,41,Male,AI/ML Engineer,"machine learning, python, deep learning, model...","Develop and deploy machine learning models, pe...",2025-06-20,"Rohan Agarwal: Hi, thanks for having me today....","Hi, for having me today. . I learn by building...","Interview questions focus on technical skills,...","[git, deep learning, python, machine learning,...",Positive,No Red Flag
8,60,INT1061,Meera Khan,39,Male,AI/ML Engineer,"machine learning, python, deep learning, model...","Develop and deploy machine learning models, pe...",2024-02-05,"Meera Khan: Hi, thanks for having me today.\nI...","Hi, for having me today. — Which tools and tec...",The candidate has around 6 years of experience...,"[git, python, detail-oriented, machine learnin...",Positive,No Red Flag
9,65,INT1066,Tara Chatterjee,34,Female,AI/ML Engineer,"machine learning, python, deep learning, model...","Develop and deploy machine learning models, pe...",2025-04-03,"Tara Chatterjee: Good afternoon, thank you for...",", for taking the time. — My strengths are mach...","In the interview transcript, focus on the cand...","[python, machine learning, cross-functional co...",Posi

In [207]:
df_final.columns

Index(['Unnamed: 0.1', 'interview_id', 'name', 'age', 'gender', 'role_applied',
       'industry_skills', 'jd', 'interview_date', 'transcript',
       'clean_transcript', 'summary', 'key_skills', 'JD_match', 'Red_Flag'],
      dtype='object')

In [208]:
df_final=df_final.drop(columns=['Unnamed: 0.1'])

In [209]:
df_final.to_csv('FINAL_DATA.csv')

## **Save_models**

In [ ]:
model.save_pretrained("/content/drive/MyDrive/saved_model/")
tokenizer.save_pretrained("/content/drive/MyDrive/saved_tokenizer/")

('saved_tokenizer/tokenizer_config.json',
 'saved_tokenizer/special_tokens_map.json',
 'saved_tokenizer/vocab.json',
 'saved_tokenizer/merges.txt',
 'saved_tokenizer/added_tokens.json',
 'saved_tokenizer/tokenizer.json')